In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
# from nltk.lm import Vocabulary
# vocabulary = Vocabulary(text)
# len(vocabulary)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 17
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])

556949
556932
['\ufeffThe Project Gute', 'The Project Guten', 'he Project Gutenb', 'e Project Gutenbe', ' Project Gutenber']


# fit

In [8]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(vocabulary)
net.to(device)

encoded = net.text2tensor(lines).to(device)
print(encoded.shape)

performance = net.fit(encoded, save_path = "checkpoints/autoregressive_lstm.pt", progress_bar = 0)
net.save_architecture("architectures/autoregressive_lstm.architecture")

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

torch.Size([556932, 19])
Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   1.3625 |     37.194 |     0.8
    2 |   1.0773 |     30.880 |     1.7
    3 |   1.0234 |     29.590 |     2.5
    4 |   0.9925 |     28.860 |     3.4
    5 |   0.9729 |     28.371 |     4.2


In [9]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

,epoch,train_loss,train_error_rate,minutes,learning_rate,weight_decay,model,embedding_dimension,hidden_units,layers,dropout,parameters
0,1,1.362476,37.194238,0.821268,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
1,2,1.077298,30.879623,1.658315,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
2,3,1.023353,29.589868,2.506511,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
3,4,0.992494,28.860186,3.352504,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
4,5,0.972910,28.370848,4.199089,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335


In [10]:
# the input for testing

net.tensor2text(encoded[:5])

['<START>\ufeffThe Project Gute<END>',
 '<START>The Project Guten<END>',
 '<START>he Project Gutenb<END>',
 '<START>e Project Gutenbe<END>',
 '<START> Project Gutenber<END>']

# predict

In [11]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
idx, log_probabilities = net.predict(encoded[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335



['<START>\ufeffThe Project Gute<END>',
 '<START>The Project Guten<END>',
 '<START>he Project Gutenb<END>',
 '<START>e Project Gutenbe<END>',
 '<START> Project Gutenber<END>']

In [12]:
log_probabilities

tensor([[  0.0000, -18.1405, -18.5183, -18.9254, -19.2813],
        [  0.0000, -18.2268, -18.4837, -18.8527, -19.3396],
        [  0.0000, -18.2711, -18.5763, -19.0656, -19.8647],
        [  0.0000, -18.0455, -18.2597, -18.5316, -18.8450],
        [  0.0000, -18.0135, -18.2310, -18.2523, -18.6434]], device='cuda:0')

# greedy_search

In [13]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([0., 0., 0., 0., 0.], device='cuda:0')


['<START>\ufeffThe Project Gute<END>',
 '<START>The Project Guten<END>',
 '<START>he Project Gutenb<END>',
 '<START>e Project Gutenbe<END>',
 '<START> Project Gutenber<END>']

# sample

In [14]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([0., 0., 0., 0., 0.], device='cuda:0')


['<START>\ufeffThe Project Gute<END>',
 '<START>The Project Guten<END>',
 '<START>he Project Gutenb<END>',
 '<START>e Project Gutenbe<END>',
 '<START> Project Gutenber<END>']

# beam_search

In [15]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5], progress_bar = False)

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([[  0.0000, -18.1405, -18.5183, -18.9254, -19.2813],
        [  0.0000, -18.2268, -18.4837, -18.8527, -19.3396],
        [  0.0000, -18.2711, -18.5763, -19.0656, -19.8647],
        [  0.0000, -18.0455, -18.2597, -18.5316, -18.8450],
        [  0.0000, -18.0135, -18.2310, -18.2523, -18.6434]], device='cuda:0')
[['<START>\ufeffThe Project Gute<END>',
  '<START>\ufeffThe Project Gute<END>',
  '<START>\ufeffThe Project Gute<END>',
  '<START>\ufeffThe Project Gute<END>',
  '<START>\ufeffThe Project Gute<END>'],
 ['<START>The Project Guten<END>',
  '<START>The Project Guten<END>',
  '<START>The Project Guten<END>',
  '<START>The Project Guten<END>',
  '<START>The Project Guten<END>'],
 ['<START>he Project Gutenb<END>',
  '<START>he Project Gutenb<END>',
  '<START>he Project Gutenb<END>',
  '<START>he Project Gutenb<END>',
  '<START>he 